In [1]:
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoConfig
import evaluate
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
import wandb
import random
from frontModelCustom import frontModelDataset, data_labels


/home/leesk/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
def set_seed(seed:int = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
set_seed(42)

In [4]:
data = pd.read_csv('./genre_11_tempo_4.csv')
data.head()

,singer,song_name,index,emotion,caption,tempo,instruments,tempo(int),tempo(category),genre,sub_genre
0,"10,000 Maniacs",A Campfire Song,2,nostalgia,"The melody is mellow and soothing, with a gent...",Tempo_135.0,"['Program_-1', 'Program_16', 'Program_24', 'Pr...",135,Allegro,Rock,Alternative rock
1,101 Strings,Theme From The Godfather,3,nostalgia,Elegant and sweeping orchestral melody with a ...,Tempo_80.0,"['Program_-1', 'Program_0', 'Program_12', 'Pro...",80,Moderato,Pop,Pop
2,10cc,Dreadlock Holiday,4,excitement,"catchy and upbeat, featuring a memorable guita...",Tempo_103.0,"['Program_-1', 'Program_0']",103,Moderato,Rock,Reggae rock
3,10cc,I'm Not In Love,5,nostalgia,A catchy and memorable tune with a simple yet ...,Tempo_65.0,"['Program_-1', 'Program_0', 'Program_122', 'Pr...",65,Andante,Rock,Soft rock
4,10cc,The Things We Do for Love,6,love,"The melody is catchy and memorable, with a ble...",Tempo_111.0,"['Program_-1', 'Program_0', 'Program_16', 'Pro...",111,Moderato,Rock,Soft rock


In [5]:
emotion_list = list(data.emotion.unique())
tempo_list = list(data['tempo(category)'].unique())
genre_list = list(data['genre'].unique())
category = emotion_list + tempo_list + genre_list
category

['nostalgia',
 'excitement',
 'love',
 'anger',
 'happiness',
 'sadness',
 'calmness',
 'gratitude',
 'loneliness',
 'anticipation',
 'Allegro',
 'Moderato',
 'Andante',
 'Presto',
 'Rock',
 'Pop',
 'Dance/Electronic',
 'Jazz',
 'R&B',
 'Groove',
 'Folk',
 'Classical',
 'World',
 'Metal',
 "Children's music"]

In [6]:
id2label_category = {k:l for k, l in enumerate(category)}
label2id_category = {l:k for k, l in enumerate(category)}

In [7]:
class customDataset():
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        self.dataset = []
        datas = []
        self.labels = []
        for idx, df in tqdm(data.iterrows()):
            labels = [0. for _ in range(len(category))]
            
            datas.append(df.caption)
            labels[label2id_category[df.emotion]] = 1.
            labels[label2id_category[df['tempo(category)']]] = 1.
            labels[label2id_category[df['genre']]] = 1.
            self.labels.append(labels)
        self.dataset =  tokenizer(datas,padding=True, truncation=True,max_length=512 ,return_tensors="pt").to('cuda')
        self.labels= torch.tensor(self.labels)

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.dataset.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item

In [8]:
BASE_MODEL = 'FacebookAI/roberta-large'

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
config = AutoConfig.from_pretrained(BASE_MODEL,num_labels = len(category))
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
data2 = data.copy()
emotion_data = data2.groupby('emotion').sample(frac=0.05, random_state=42)
tempo_data = data2.groupby('tempo(category)').sample(frac=0.05, random_state=42)
genre_data = data2.groupby('genre').sample(frac=0.05, random_state=42)
index_total = set(emotion_data.index) | set(tempo_data.index) | set(genre_data.index)
valid_data = data2.iloc[list(index_total)]
train_data = data2.drop(list(index_total)).sample(frac=1, random_state=42)

In [10]:

dataset_train = customDataset(train_data, tokenizer =tokenizer)
dataset_valid = customDataset(valid_data, tokenizer =tokenizer)

8552it [00:00, 16981.33it/s]
1419it [00:00, 17350.68it/s]


In [11]:
def get_preds_from_logits(logits):
    ret = np.zeros(logits.shape)
    
    best_class = np.argmax(logits, axis=1)
    ret[list(range(len(ret))), best_class] = 1
    
    return ret

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    print(logits)
    final_metrics = {}
    first = len(emotion_list)
    second = first+len(tempo_list)
    third = second + len(tempo_list)
    # Deduce predictions from logits
    logits_emotion = logits[:first]
    logits_emotion = get_preds_from_logits(logits_emotion)
    
    logits_tempo = logits[first:second]
    logits_tempo = get_preds_from_logits(logits_tempo)

    logits_genre = logits[second:]
    logits_genre = get_preds_from_logits(logits_genre)
    
    labels_emotion = logits[:first]
    labels_tempo = logits[first:second]
    labels_genre = logits[second:]
    # Get f1 metrics for global scoring. Notice that f1_micro = accuracy
    final_metrics["f1_emotion"] = f1_score(labels_emotion, logits_emotion, average="micro")
    
    # Get f1 metrics for causes
    final_metrics["f1_tempo"] = f1_score(labels_tempo, logits_tempo, average="micro")
    

    # The global f1_metrics
    final_metrics["f1_genre"] = f1_score(labels_genre, logits_genre, average="micro")

    final_metrics['fi_total'] = (final_metrics["f1_emotion"] + final_metrics["f1_tempo"] + final_metrics["f1_genre"])
    return final_metrics

In [13]:
# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.pop(f"labels")
#         outputs = model(**inputs)
#         logits = outputs[0]
#         print('compute_loss logits : ',logits)
#         # global_score_loss = torch.nn.functional.cross_entropy(logits, labels)
#         cause_loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, labels)
        
#         return (cause_loss, outputs) if return_outputs else cause_loss

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:

# wandb.init(project="Final project", entity="sanggang", name = "no_add_layer"+BASE_MODEL)

training_args = TrainingArguments(

   output_dir="my_awesome_model",
   save_steps=300,
   eval_steps = 300, 
   warmup_steps=500,
   logging_steps=100,
   learning_rate=5e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=6,
   weight_decay=0.01,
   evaluation_strategy='steps',
   load_best_model_at_end = True,
   save_total_limit = 2,
   # report_to="wandb",
   metric_for_best_model='fi_total',
   # run_name=BASE_MODEL, 
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=dataset_train,
   eval_dataset=dataset_valid,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
